In [1]:
# Import packages
import os
from typing import List

import pandas as pd
import numpy as np
from openpyxl import load_workbook
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from joblib import dump, load
import matplotlib.pyplot as plt
import shap
import seaborn as sns
from hydroeval import evaluator, nse

sns.set_theme()

In [2]:
# Set project directory
def set_project_dir():
    try:
        os.chdir(r'\\export.hpc.ut.ee\gis\flow_swat_ml_paper')
    except OSError:
        os.chdir('D:/flow_swat_ml_paper')


# Get Excel file based on country code
def get_excel_file(country_code: str) -> str:
    return f'ml/{country_code}/source_data/{country_code}.xlsx'


# Add missing dates to DataFrame
def add_missing_dates(df: pd.DataFrame) -> pd.DataFrame:
    min_date = df['Date'].min().strftime('%Y-%m-%d')
    max_date = df['Date'].max().strftime('%Y-%m-%d')
    index = pd.date_range(min_date, max_date)
    df = df.set_index('Date').reindex(index, fill_value=np.nan)
    df = df.reset_index().rename(columns={'index': 'Date'})
    return df


# Get aggregation function based on time interval
def get_agg_func(variable: str, time_interval: str):
    if variable in ['Q', 'Tmax', 'Tmin']:
        return 'mean'
    elif variable == 'Pcp':
        if time_interval.lower() == 'd':
            return 'mean'
        elif time_interval.lower() == 'm':
            return 'sum'


# Read flow data from Excel file
def read_flow_data(country_code: str, time_interval: str) -> pd.DataFrame:
    excel_file = get_excel_file(country_code)
    flow_df = pd.read_excel(excel_file, sheet_name='Flow')
    flow_df = flow_df.dropna(subset=['Date']).reset_index(drop=True)
    flow_df = add_missing_dates(flow_df)
    variable = 'Q'
    agg_func = get_agg_func(variable, time_interval)
    flow_df = flow_df.set_index('Date')
    flow_df = flow_df.resample(time_interval).agg(agg_func).reset_index().round(1)
    flow_df = flow_df.rename(columns={flow_df.columns[1]: f'{variable}_{time_interval.lower()}'})
    return flow_df


# Read predictor variable data from Excel file
def read_pred_var_data(country_code: str, time_interval: str, na_values=None) -> pd.DataFrame:
    excel_file = get_excel_file(country_code)
    sheet_names = load_workbook(excel_file, read_only=True).sheetnames
    pred_var_df_list = []
    invalid_df_list = []
    variables = ['Pcp', 'Tmax', 'Tmin']
    for sheet_name in sheet_names:
        if sheet_name != 'Flow':
            df = pd.read_excel(excel_file, sheet_name=sheet_name, na_values=na_values)
            df = df.rename(
                columns={df.columns[1]: variables[0], df.columns[2]: variables[1], df.columns[3]: variables[2]}
            )
            df = df.dropna(subset=['Date']).reset_index(drop=True)
            df = add_missing_dates(df)
            df['Station'] = sheet_name
            # Get indices of dates with invalid minimum temperature values
            indices = df.loc[df['Tmax'] < df['Tmin']].index
            # Replace temperature values on those dates with NaN
            if len(indices) > 0:
                invalid_df_list.append(df.loc[indices])
                df.loc[indices, 'Tmax'] = np.nan
                df.loc[indices, 'Tmin'] = np.nan
            df = df.set_index('Date')
            # Fill missing temperature values
            if df['Tmax'].isna().sum() > 0:
                df['Tmax'] = df['Tmax'].interpolate('time')
            if df['Tmin'].isna().sum() > 0:
                df['Tmin'] = df['Tmin'].interpolate('time')
            agg_func_dict = {}
            column_name_dict = {}
            for variable in variables:
                agg_func_dict[variable] = get_agg_func(variable, time_interval)
                column_name_dict[variable] = f'{variable}_{time_interval}'
            df = df.resample(time_interval).agg(agg_func_dict).reset_index().round(1)
            df = df.rename(columns=column_name_dict)
            pred_var_df_list.append(df)
    if len(invalid_df_list) > 0:
        invalid_df = pd.concat(invalid_df_list).reset_index(drop=True)
        invalid_df.to_csv(f'ml/{country_code}/source_data/{country_code}_Tmin_invalid.csv', index=False)
    pred_var_df = pd.concat(pred_var_df_list).groupby('Date').agg('mean').reset_index().round(1)
    return pred_var_df


# Get DataFrame with lags
def calc_lags(
        df: pd.DataFrame, variables: List[str], time_interval: str, lag_width: int, n_lags: int
) -> pd.DataFrame:
    lags = df.copy()
    for variable in variables:
        for i in range(1, lag_width + n_lags, lag_width):
            lags[f'{variable}_{time_interval}-{i}'] = lags[f'{variable}_{time_interval}'].shift(i)
    return lags


# Get number of lags based on time interval
def get_n_lags(time_interval: str) -> int:
    if time_interval.lower() == 'd':
        return 28
    elif time_interval.lower() == 'm':
        return 12


# Get DataFrame with rolling aggregates
def calc_rolling_aggs(
        df: pd.DataFrame, variables: List[str], time_interval: str, window_width: int, n_windows: int,
        agg_func_list: List[str]
) -> pd.DataFrame:
    rolling_aggs = df.copy()
    for variable in variables:
        for i in range(window_width, window_width * n_windows + window_width, window_width):
            for agg_func in agg_func_list:
                agg_col = f'{variable}_{time_interval}-{i}_rolling_{agg_func}'
                rolling_aggs[agg_col] = rolling_aggs[f'{variable}_{time_interval}'].rolling(i).agg(agg_func).round(1)
    return rolling_aggs


# Get window width based on time interval
def get_window_width(time_interval: str) -> int:
    if time_interval.lower() == 'd':
        return 7
    elif time_interval.lower() == 'm':
        return 3


# Get DataFrame with leads
def calc_leads(
        df: pd.DataFrame, variables: List[str], time_interval: str, lead_width: int, n_leads: int
) -> pd.DataFrame:
    leads = df.copy()
    for variable in variables:
        for i in range(1, lead_width + n_leads, lead_width):
            leads[f'{variable}_{time_interval}+{i}'] = leads[f'{variable}_{time_interval}'].shift(-i)
    return leads


# Get path of model directory
def get_model_dir(country_code: str, target: str, feat_set: str) -> str:
    model_dir = f'ml/{country_code}/models/rf/{country_code}_{target}_{feat_set}'
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    return model_dir


# Get catchment name based on country code
def get_catchment_name(country_code: str) -> str:
    if country_code == 'ESP':
        return 'Argos'
    elif country_code == 'EST':
        return 'Porijõgi'
    elif country_code == 'ETH':
        return 'Rib'
    elif country_code == 'USA':
        return 'Bald Eagle'


# Get target based on feature set
def get_target(feat_set: str) -> str:
    time_interval = feat_set[-1]
    if time_interval == 'd':
        return f'Q_{time_interval}+1'
    elif time_interval == 'm':
        return f'Q_{time_interval}+1'


# Get feature set list based on target
def get_feat_set(target: str) -> str:
    if target == 'Q_d+1':
        return ['FS1_d', 'FS2_d', 'FS3_d']
    elif target == 'Q_m+1':
        return ['FS1_m', 'FS2_m', 'FS3_m']

In [3]:
# Change working directory
working_dir = r'\\export.hpc.ut.ee\gis\flow_swat_ml_paper'
os.chdir(working_dir)

In [4]:
excel_file = 'swat/SWAT_results.xlsx'

# Daily models

In [5]:
target = 'Q_d+1'
country_codes = ['ESP', 'EST', 'ETH', 'USA']
feat_set_list = ['FS1_d', 'FS2_d', 'FS3_d']
# test_size_list = [0.5, 0.4, 0.3, 0.2, 0.1]
test_size_list = [0.5]
time_interval = 'D'

In [6]:
results_list = []

for country_code in country_codes:
    
    # Read SWAT results
    sheet_name = f'{country_code}_{time_interval}'
    swat_results = pd.read_excel(excel_file, sheet_name=sheet_name)

    # Get start and end of SWAT time series
    swat_min_date = swat_results[swat_results.columns[0]].min()
    swat_max_date = swat_results[swat_results.columns[0]].max()
    
    for feat_set in feat_set_list:
        model_dir = get_model_dir(country_code, target, feat_set)
        
        for test_size in test_size_list:
            test_size_int = test_size_int = int(test_size * 100)

            # Get indices of training samples
            train_indices = pd.read_csv(
                f'{model_dir}/{country_code}_{target}_{feat_set}_feat_train_{test_size_int}.csv', usecols=['Index']
            )['Index'].values

            # Get indices of test samples
            test_indices = pd.read_csv(
                f'{model_dir}/{country_code}_{target}_{feat_set}_feat_test_{test_size_int}.csv', usecols=['Index']
            )['Index'].values

            # Read RF results
            obs_vs_pred = pd.read_csv(f'{model_dir}/{country_code}_{target}_{feat_set}_obs_vs_pred_{test_size_int}.csv', parse_dates=['Date'])
            obs_vs_pred['Index'] = obs_vs_pred.index

            # Drop rows outside of the SWAT study period
            obs_vs_pred = obs_vs_pred.loc[(swat_min_date <= obs_vs_pred['Date']) & (obs_vs_pred['Date'] <= swat_max_date)]

            # Training set
            end_index_train = train_indices[-1]
            obs_vs_pred_train = obs_vs_pred.loc[:end_index_train, :]
            obs_train = obs_vs_pred_train[target]
            pred_train = obs_vs_pred_train[f'{target}_pred']
            rmse_train = round(mean_squared_error(obs_train, pred_train, squared=False), 2)
            r2_train = round(r2_score(obs_train, pred_train), 2)
            nse_train = round(evaluator(nse, pred_train, obs_train)[0], 2)

            # Test set
            start_index_test = test_indices[0]
            obs_vs_pred_test = obs_vs_pred.loc[start_index_test:, :]
            obs_test = obs_vs_pred_test[target]
            pred_test = obs_vs_pred_test[f'{target}_pred']
            rmse_test = round(mean_squared_error(obs_test, pred_test, squared=False), 2)
            r2_test = round(r2_score(obs_test, pred_test), 2)
            nse_test = round(evaluator(nse, pred_test, obs_test)[0], 2)

            # Append results to list
            results = {
                'country_code': country_code,
                'target': target,
                'feat_set': feat_set,
                'model_version': f'{target}_{feat_set}',
                'test_size_int': test_size_int,
                'nse_train': nse_train,
                'nse_test': nse_test,
                'rmse_train': rmse_train,
                'rmse_test': rmse_test,
                'r2_train': r2_train,
                'r2_test': r2_test
            }
            df_results = pd.DataFrame([results.values()], columns=results.keys())
            results_list.append(df_results)
results_df = pd.concat(results_list).reset_index(drop=True)

In [7]:
results_df

,country_code,target,feat_set,model_version,test_size_int,nse_train,nse_test,rmse_train,rmse_test,r2_train,r2_test
0,ESP,Q_d+1,FS1_d,Q_d+1_FS1_d,50,0.90,-0.01,0.15,0.65,0.90,-0.01
1,ESP,Q_d+1,FS2_d,Q_d+1_FS2_d,50,0.92,0.20,0.14,0.58,0.92,0.20
2,ESP,Q_d+1,FS3_d,Q_d+1_FS3_d,50,0.90,0.24,0.15,0.57,0.90,0.24
3,EST,Q_d+1,FS1_d,Q_d+1_FS1_d,50,0.99,-0.67,0.25,1.54,0.99,-0.67
4,EST,Q_d+1,FS2_d,Q_d+1_FS2_d,50,0.99,0.82,0.23,0.50,0.99,0.82
5,EST,Q_d+1,FS3_d,Q_d+1_FS3_d,50,0.99,0.85,0.23,0.46,0.99,0.85
6,ETH,Q_d+1,FS1_d,Q_d+1_FS1_d,50,0.99,0.81,2.81,10.74,0.99,0.81
7,ETH,Q_d+1,FS2_d,Q_d+1_FS2_d,50,0.98,0.88,3.30,8.50,0.98,0.88
8,ETH,Q_d+1,FS3_d,Q_d+1_FS3_d,50,0.99,0.88,2.89,8.49,0.99,0.88
9,USA,Q_d+1,FS1_d,Q_d+1_FS1_d,50,0.91,-1.02,4.92,17.62,0.91,-1.02


In [8]:
results_df.to_csv(fr'\\export.hpc.ut.ee\gis\flow_swat_ml_paper\ml\{target}_rf_metrics.csv', index=False)

# Monthly models

In [9]:
target = 'Q_m+1'
country_codes = ['ESP', 'EST', 'ETH', 'USA']
feat_set_list = ['FS1_m', 'FS2_m', 'FS3_m']
# test_size_list = [0.5, 0.4, 0.3, 0.2, 0.1]
test_size_list = [0.5]
time_interval = 'M'

In [13]:
results_list = []

for country_code in country_codes:
    
    # Read SWAT results
    sheet_name = f'{country_code}_{time_interval}'
    swat_results = pd.read_excel(excel_file, sheet_name=sheet_name)
    
    # Shift date to the end of the month to match with RF time series
    swat_results[swat_results.columns[0]] = swat_results[swat_results.columns[0]] + pd.offsets.MonthEnd(0)

    # Get start and end of SWAT time series
    swat_min_date = swat_results[swat_results.columns[0]].min()
    swat_max_date = swat_results[swat_results.columns[0]].max()
    
    for feat_set in feat_set_list:
        model_dir = get_model_dir(country_code, target, feat_set)
        
        for test_size in test_size_list:
            test_size_int = test_size_int = int(test_size * 100)

            # Get indices of training samples
            train_indices = pd.read_csv(
                f'{model_dir}/{country_code}_{target}_{feat_set}_feat_train_{test_size_int}.csv', usecols=['Index']
            )['Index'].values

            # Get indices of test samples
            test_indices = pd.read_csv(
                f'{model_dir}/{country_code}_{target}_{feat_set}_feat_test_{test_size_int}.csv', usecols=['Index']
            )['Index'].values

            # Read RF results
            obs_vs_pred = pd.read_csv(f'{model_dir}/{country_code}_{target}_{feat_set}_obs_vs_pred_{test_size_int}.csv', parse_dates=['Date'])
            obs_vs_pred['Index'] = obs_vs_pred.index

            # Drop rows outside of the SWAT study period
            obs_vs_pred = obs_vs_pred.loc[(swat_min_date <= obs_vs_pred['Date']) & (obs_vs_pred['Date'] <= swat_max_date)]

            # Training set
            end_index_train = train_indices[-1]
            obs_vs_pred_train = obs_vs_pred.loc[:end_index_train, :]
            
            obs_vs_pred_train = obs_vs_pred_train.loc[(swat_min_date <= obs_vs_pred_train['Date']) & (obs_vs_pred_train['Date'] <= swat_max_date)]
            
            rf_train_min_date = obs_vs_pred_train['Date'].dt.strftime('%Y-%m-%d').min()
            rf_train_max_date = obs_vs_pred_train['Date'].dt.strftime('%Y-%m-%d').max()
            
            obs_train = obs_vs_pred_train[target]
            pred_train = obs_vs_pred_train[f'{target}_pred']
            rmse_train = round(mean_squared_error(obs_train, pred_train, squared=False), 2)
            r2_train = round(r2_score(obs_train, pred_train), 2)
            nse_train = round(evaluator(nse, pred_train, obs_train)[0], 2)

            # Test set
            start_index_test = test_indices[0]
            obs_vs_pred_test = obs_vs_pred.loc[start_index_test:, :]
            
            obs_vs_pred_test = obs_vs_pred_test.loc[(swat_min_date <= obs_vs_pred_test['Date']) & (obs_vs_pred_test['Date'] <= swat_max_date)]
            
            rf_test_min_date = obs_vs_pred_test['Date'].dt.strftime('%Y-%m-%d').min()
            rf_test_max_date = obs_vs_pred_test['Date'].dt.strftime('%Y-%m-%d').max()
            
            obs_test = obs_vs_pred_test[target]
            pred_test = obs_vs_pred_test[f'{target}_pred']
            rmse_test = round(mean_squared_error(obs_test, pred_test, squared=False), 2)
            r2_test = round(r2_score(obs_test, pred_test), 2)
            nse_test = round(evaluator(nse, pred_test, obs_test)[0], 2)
            
            print(country_code)
            print(rf_train_min_date, rf_train_max_date, rf_test_min_date, rf_test_max_date)

            # Append results to list
            results = {
                'country_code': country_code,
                'target': target,
                'feat_set': feat_set,
                'model_version': f'{target}_{feat_set}',
                'test_size_int': test_size_int,
                'nse_train': nse_train,
                'nse_test': nse_test,
                'rmse_train': rmse_train,
                'rmse_test': rmse_test,
                'r2_train': r2_train,
                'r2_test': r2_test
            }
            df_results = pd.DataFrame([results.values()], columns=results.keys())
            results_list.append(df_results)
results_df = pd.concat(results_list).reset_index(drop=True)

ESP
2006-01-31 2010-12-31 2011-01-31 2016-12-31
ESP
2006-01-31 2010-12-31 2011-01-31 2016-12-31
ESP
2006-01-31 2010-12-31 2011-01-31 2016-12-31
EST
2009-01-31 2013-12-31 2014-01-31 2019-12-31
EST
2009-01-31 2013-12-31 2014-01-31 2019-12-31
EST
2009-01-31 2013-12-31 2014-01-31 2019-12-31
ETH
1997-01-31 2001-12-31 2002-01-31 2007-12-31
ETH
1997-01-31 2001-12-31 2002-01-31 2007-12-31
ETH
1997-01-31 2001-12-31 2002-01-31 2007-12-31
USA
2002-01-31 2006-12-31 2007-01-31 2012-12-31
USA
2002-01-31 2006-12-31 2007-01-31 2012-12-31
USA
2002-01-31 2006-12-31 2007-01-31 2012-12-31


In [12]:
results_df

,country_code,target,feat_set,model_version,test_size_int,nse_train,nse_test,rmse_train,rmse_test,r2_train,r2_test
0,ESP,Q_m+1,FS1_m,Q_m+1_FS1_m,50,0.91,0.27,0.09,0.28,0.91,0.27
1,ESP,Q_m+1,FS2_m,Q_m+1_FS2_m,50,0.92,0.34,0.08,0.27,0.92,0.34
2,ESP,Q_m+1,FS3_m,Q_m+1_FS3_m,50,0.92,0.38,0.08,0.26,0.92,0.38
3,EST,Q_m+1,FS1_m,Q_m+1_FS1_m,50,0.91,0.13,0.52,0.82,0.91,0.13
4,EST,Q_m+1,FS2_m,Q_m+1_FS2_m,50,0.91,0.01,0.50,0.88,0.91,0.01
5,EST,Q_m+1,FS3_m,Q_m+1_FS3_m,50,0.91,0.05,0.51,0.86,0.91,0.05
6,ETH,Q_m+1,FS1_m,Q_m+1_FS1_m,50,0.96,0.87,4.35,7.74,0.96,0.87
7,ETH,Q_m+1,FS2_m,Q_m+1_FS2_m,50,0.98,0.76,3.47,10.43,0.98,0.76
8,ETH,Q_m+1,FS3_m,Q_m+1_FS3_m,50,0.97,0.88,3.58,7.44,0.97,0.88
9,USA,Q_m+1,FS1_m,Q_m+1_FS1_m,50,0.89,0.29,2.58,6.53,0.89,0.29


In [12]:
results_df.to_csv(fr'\\export.hpc.ut.ee\gis\flow_swat_ml_paper\ml\{target}_rf_metrics.csv', index=False)